In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
import dataset

prostate_whole_dataset = dataset.ProstatDS("dataset/prostate.txt")

In [ ]:
test, val, train = dataset.dataset_split(prostate_whole_dataset, [0.2, 0.1])
train.normalize()
val.normalize(train.norm_values)
_ = test.normalize(train.norm_values)

# Gradient Check

In [ ]:
from gradient_check import eval_numeric_gradient
from gradient_check import rel_error
import models

sample, _ = dataset.dataset_split(train, 40.0 / train.num_samples)
X = sample.X.astype("float64")

for reg in [0, 0.1]:
  model = models.TwoLayerNet(train.num_features, 20, reg=reg)
  
  # Post-initialization: Make the 'b' params random values as well:
  for key in model.params.keys():
    p = model.params[key].astype("float64")
    if key.startswith('b'):
      p = 0.01 * np.random.randn(*p.shape)
    model.params[key] = p
  print("Testing with reg=%.2e:" % reg)

  for key in model.params.keys():
    f = lambda : model.loss(X, sample.y)[0]
    grad_n = eval_numeric_gradient(f, model.params[key])
    grad_a = model.loss(sample.X, sample.y)[1][key]
    error = rel_error(grad_n, grad_a)
    print("%s, error=%.1e" % (key, error))


In [ ]:
def accuracy(model):
  y_train = model.loss(train.X)
  acc_train = np.mean(y_train == train.y)
  y_val = model.loss(val.X)
  acc_val = np.mean(y_val == val.y)
  print("Accuracy: Training = %.1f%%, Validation = %.1f%%" % (acc_train * 100, acc_val * 100))

In [ ]:
H = 18
reg = 1e-5
learning_rate = 3
learning_rate_decay = 0.99
model = models.TwoLayerNet(train.num_features, H, reg=reg)

for i in range(300):
  loss, grad = model.loss(train.X, train.y)
  for key in grad.keys():
    model.params[key] -= learning_rate * grad[key]
  if i % 10 == 0:
    learning_rate *= learning_rate_decay
  if i % 10 == 0:
    print("[%4d]: loss=%f" % (i, loss))
    accuracy(model)
  


In [ ]:
y_pred = model.loss(test.X)
print("Test Accuracy: %.2f%%" % (np.mean(y_pred == test.y)))